In [83]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [52]:
data = pd.read_csv('diamonds.csv')
data = data.drop('Unnamed: 0', axis=1)
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


In [54]:
categorical = ['cut','color','clarity']

In [55]:
lre = LabelEncoder()

In [56]:
for el in categorical:
    data[el] = lre.fit_transform(data[el])

In [67]:
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75


In [69]:
X = data.drop('price', axis=1)
y = data['price']

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=0.3)

In [121]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [123]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes=[8,4,2], dropout_rate=0.25):
        super(MLPClassifier, self).__init__()
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size

        layers.append(nn.Linear(prev_size, 1))
        layers.append(nn.Sigmoid())

        self.networks = nn.Sequential(*layers)

    def forward(self, x):
        return self.networks(x)

In [125]:
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1,1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1,1)

In [127]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [129]:
def train_mlp(model, train_loader, num_epochs=100, learning_rate=0.001):
    criterion = nn.BCELoss() #Бинарная кросс-энтропия для задачи классификации
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 1e-5) #создаем опитимизатор Adam с коэф регуляризации
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1) #планировщик скорости обучения

    train_losses = [] #значения функции потерь на каждой эпохе
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad() #обнуление градиентов у всех параметров перед вычислением новых
            outputs = model(batch_X) #происходит forward pass ->
            loss = criterion(outputs, batch_y) #вычисляется функция потерь между y и ^y
            loss.backward() #обратное распространение ошибки backward pass
            optimizer.step() #обновление параметров модели
            epoch_loss += loss.item() #потеря за БАТЧ добавляется к потере за ЭПОХУ

        scheduler.step() #обновление скорости обучения
        avg_loss = epoch_loss / len(train_loader) #вычисляем среднюю потерю за ЭПОХУ
        train_losses.append(avg_loss) #сохраняем 

        if (epoch + 1) % 20 == 0:
            print(f'Epoch loss = {avg_loss:.4f}') #будем выводить среднюю потерю за каждую 20 ЭПОХУ 

    return train_losses

In [131]:
mlp_model = MLPClassifier(input_size=X_train_scaled.shape[1])

In [135]:
train_loses = train_mlp(mlp_model, train_loader, num_epochs=200)

RuntimeError: all elements of target should be between 0 and 1